In [3]:
import pandas as pd

In [4]:
df_device = pd.read_csv('./device.csv')

In [5]:
df_device.columns

Index(['id', 'date', 'user', 'pc', 'activity'], dtype='object')

In [11]:
df_device.describe()

,id,date,user,pc,activity
count,417588,417588,417588,417588,417588
unique,417588,412105,218,215,2
top,{I5K5-M8HD47HI-6694GOIH},10/29/2010 09:08:19,PCM0369,PC-2094,Connect
freq,1,3,8085,8085,211663


In [12]:
df_device.sort_values('date', inplace=True)

In [6]:
df_device.iloc[0]

id          {I5K5-M8HD47HI-6694GOIH}
date             01/02/2010 07:13:39
user                         NLR0174
pc                           PC-8272
activity                     Connect
Name: 0, dtype: object

In [13]:
df_firstActivity = df_device.groupby(['user','activity'])['date'].min().reset_index()
df_firstActivity['date'] = pd.to_datetime(df_firstActivity['date'])
df_firstActivity['calendarDay'] = df_firstActivity['date'].dt.date
df_firstActivity['dateString'] = df_firstActivity['calendarDay'].apply(lambda x: x.strftime('%Y%m%d'))
df_firstActivity['userDate'] = df_firstActivity.apply(lambda x: x['user'] + '_' + x['dateString'], axis=1)
df_firstActivity = df_firstActivity[['userDate','activity']]
df_firstActivity['firstActivity'] = 1
#df_firstActivity.to_pickle('firstActivity.pkl')

In [14]:
df_firstActivity.head(5)

,userDate,activity,firstActivity
0,AAS0850_20100104,Connect,1
1,AAS0850_20100104,Disconnect,1
2,ABH0821_20110102,Connect,1
3,ABH0821_20110102,Disconnect,1
4,ACR0100_20110104,Connect,1


In [15]:
df_hasConnect = df_device[df_device['activity'] == 'Connect'].copy()
df_hasConnect['date'] = pd.to_datetime(df_hasConnect['date'])
df_hasConnect['calendarDay'] = df_hasConnect['date'].dt.date
df_hasConnect['dateString'] = df_hasConnect['calendarDay'].apply(lambda x: x.strftime('%Y%m%d'))
df_hasConnect['userDate'] = df_hasConnect.apply(lambda x: x['user'] + '_' + x['dateString'], axis=1)
df_hasConnect = df_hasConnect[['userDate']]
df_hasConnect['hasConnect'] = 1
df_hasConnect = df_hasConnect.groupby('userDate').sum().reset_index()
#df_hasConnect.to_pickle('hasConnect.pkl')

In [16]:
df_hasConnect.head(5)

,userDate,hasConnect
0,AAS0850_20100104,1
1,AAS0850_20100106,2
2,AAS0850_20100108,2
3,AAS0850_20100112,1
4,AAS0850_20100113,1


In [26]:
# no user connected a usb into multiple computers
df_test = df_device[df_device['activity'] == 'Connect'].copy()
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['calendarDay'] = df_test['date'].dt.date
df_test['dateString'] = df_test['calendarDay'].apply(lambda x: x.strftime('%Y%m%d'))
df_test['userDate'] = df_test.apply(lambda x: x['user'] + '_' + x['dateString'], axis=1)
df_test = df_test.groupby('userDate').nunique('pc')
df_test[df_test['pc'] != 1]

,id,date,user,pc,activity,calendarDay,dateString
userDate,,,,,,,


In [30]:
# there are some computer that have multiple usb:user per day
df_test = df_device[df_device['activity'] == 'Connect'].copy()
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['calendarDay'] = df_test['date'].dt.date
df_test['dateString'] = df_test['calendarDay'].apply(lambda x: x.strftime('%Y%m%d'))
df_test['userDate'] = df_test.apply(lambda x: x['user'] + '_' + x['dateString'], axis=1)
df_test = df_test.groupby(['pc','dateString']).nunique('user')
df_test[df_test['user'] != 1]

id  date  user  activity  calendarDay  userDate
pc      dateString                                                 
PC-0583 20100104     5     5     2         1            1         2
        20100108     3     3     2         1            1         2
        20100111     4     4     2         1            1         2
        20100112     3     3     2         1            1         2
        20100113     5     5     2         1            1         2
...                 ..   ...   ...       ...          ...       ...
PC-3207 20110419     6     6     2         1            1         2
        20110420    11    11     2         1            1         2
        20110421     8     8     2         1            1         2
        20110510     9     9     2         1            1         2
        20110512    10    10     2         1            1         2

[309 rows x 6 columns]